In [8]:
import os

os.getcwd()

'C:\\Users\\Eeeta\\GeeseHacks'

In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="eGjHxmLCAIMYJ89pbtFB")
project = rf.workspace("eeetan").project("road-names-y55wg")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to road-names-3 in yolov8:: 100%|██████████| 1730/1730 [00:00<00:00, 3382.59it/s]


In [2]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Eeeta\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 47.2MB/s]


In [15]:
results = model.train(data="C://Users/Eeeta/Geesehacks/road-names-3/data.yaml", epochs=100, imgsz=640,batch=8)

Ultralytics 8.3.67  Python-3.11.7 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C://Users/Eeeta/Geesehacks/road-names-3/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, s

100%|██████████| 5.35M/5.35M [00:00<00:00, 35.9MB/s]


AMP: checks passed 


train: Scanning C:\Users\Eeeta\GeeseHacks\road-names-3\train\labels... 747 images, 0 backgrounds, 0 corrupt: 100%|██████████| 747/747 [00:00<00:00, 1067.25it/s]

train: New cache created: C:\Users\Eeeta\GeeseHacks\road-names-3\train\labels.cache
WARNING  Box and segment counts should be equal, but got len(segments) = 507, len(boxes) = 1059. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

    100/100       1.2G     0.4617     0.2959     0.9436          4        640: 100%|██████████| 94/94 [00:05<00:00, 18.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00, 17.85it/s]

                   all         75        104        0.9       0.95      0.927      0.663



100 epochs completed in 0.168 hours.
Optimizer stripped from runs\detect\train8\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train8\weights\best.pt, 6.3MB

Validating runs\detect\train8\weights\best.pt...
Ultralytics 8.3.67  Python-3.11.7 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00, 10.48it/s]


                   all         75        104      0.898      0.942      0.941      0.677
Speed: 0.3ms preprocess, 1.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\train8


In [17]:
trained = YOLO("best.pt")

In [18]:
results = trained.track(source=0, stream=True, tracker='bytetrack.yaml', show=True)

In [20]:
import cv2

# Iterate over the results from the generator
for result in results:
    # Get the frame with bounding boxes and other annotations
    annotated_frame = result.plot()

    # Display the annotated frame in a window
    cv2.imshow('YOLO ByteTrack Webcam', annotated_frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cv2.destroyAllWindows()

0: 480x640 (no detections), 246.2ms
0: 480x640 (no detections), 4.2ms
0: 480x640 (no detections), 6.1ms
0: 480x640 (no detections), 21.8ms
0: 480x640 (no detections), 8.0ms
0: 480x640 (no detections), 4.6ms
0: 480x640 (no detections), 5.1ms
0: 480x640 (no detections), 15.5ms
0: 480x640 (no detections), 5.2ms
0: 480x640 (no detections), 8.5ms
0: 480x640 (no detections), 7.3ms
0: 480x640 (no detections), 4.1ms
0: 480x640 (no detections), 5.2ms
0: 480x640 (no detections), 5.7ms
0: 480x640 (no detections), 14.0ms
0: 480x640 (no detections), 6.8ms
0: 480x640 (no detections), 8.3ms
0: 480x640 (no detections), 5.1ms
0: 480x640 (no detections), 9.2ms
0: 480x640 (no detections), 7.9ms
0: 480x640 (no detections), 5.6ms
0: 480x640 (no detections), 12.0ms
0: 480x640 (no detections), 8.7ms
0: 480x640 (no detections), 4.5ms
0: 480x640 (no detections), 10.2ms
0: 480x640 (no detections), 5.2ms
0: 480x640 (no detections), 6.8ms
0: 480x640 (no detections), 8.1ms
0: 480x640 (no detections), 4.2ms
0: 480x

KeyboardInterrupt: 

In [21]:
import cv2

# Open the default camera
cam = cv2.VideoCapture(0)

# Get the default frame width and height
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (frame_width, frame_height))

while True:
    ret, frame = cam.read()

    # Write the frame to the output file
    out.write(frame)

    # Display the captured frame
    cv2.imshow('Camera', frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break

# Release the capture and writer objects
cam.release()
out.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 